In [1]:
# read jsonl

import json
from datasets import load_dataset
from verl.utils.reward_score.prime_math import compute_score
# from verl.workers.reward_manager import PrimeRewardManager

/mbz/users/shibo.hao/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-12 22:39:16,040	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [17]:
from verl.utils.reward_score.prime_math import compute_score, match_answer, grade_answer

In [2]:

data = []
for node_id in range(64):
    for chunk_id in range(4):
        with open(f'../sgl_inference_results/big-math-rl-verified_problems/node_{node_id}/batch_results_chunk_{chunk_id}.jsonl', 'r') as f:
            for id, line in enumerate(f):
                if id >= 64:
                    break
                data.append(json.loads(line))
d_ = [(int(d['custom_id'].split('-')[1]), d['response']['body']['choices']['message']['content']) for d in data]

In [3]:
from datasets import load_dataset
dataset = load_dataset('SynthLabsAI/Big-Math-RL-Verified', split='train')

In [4]:

from tqdm import tqdm
saved_data = {}
for d in tqdm(d_):
    if d[0] not in saved_data:
        saved_data[d[0]] = {
            "question": dataset['problem'][d[0]],
            "answer": dataset['answer'][d[0]],
            "model_outputs": [d[1]]
        }
    else:
        saved_data[d[0]]['model_outputs'].append(d[1])

100%|██████████| 16384/16384 [02:23<00:00, 114.33it/s]


In [5]:
json.dump(saved_data, open('sampled_data.json', 'w'))

In [12]:
saved_data[0]['model_outputs']

32

In [6]:
import requests

url = f"http://localhost:30000/v1/chat/completions"

input_template = """You are a teacher and your task is to grade the student's answer with the reference answer.

Question: [QUESTION]

Student's Answer: [STUDENT'S ANSWER]

Reference Answer: [REFERENCE ANSWER]

You only need to refer to the reference answer to grade the student's answer. Sometimes the student's answer is expressed in a different way from the reference answer, but the meaning is the same, and you should still consider it correct. If they are not equivalent in mathematical sense, you should consider it incorrect.

Please provide a brief explanation (1-2 sentences) of your grading process and put your final grade in the following format:

Final Grade: CORRECT or INCORRECT
"""

In [10]:
q = "Solve the inequality system: $\left\{\begin{array}{c}3x-1>x+1\\ \frac{4x-5}{3}\leq x\end{array}\right.$."
ref = "1 < x \\leq 5"
answer = "(1, 5)"
inputs = input_template.replace("[QUESTION]", q).replace("[STUDENT'S ANSWER]", answer).replace("[REFERENCE ANSWER]", ref)
# print(inputs)
data = {
    "model": "Qwen/Qwen2.5-7B-Instruct",
    "messages": [{"role": "user", "content": inputs}],
}
response = requests.post(url, json=data)
print(not "INCORRECT" in response.json()['choices'][0]['message']['content'])

<>:1: SyntaxWarning: invalid escape sequence '\l'
<>:1: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_1956799/3900173001.py:1: SyntaxWarning: invalid escape sequence '\l'
  q = "Solve the inequality system: $\left\{\begin{array}{c}3x-1>x+1\\ \frac{4x-5}{3}\leq x\end{array}\right.$."


True


In [27]:
ground_truth = saved_data[0]['answer']

for response_id in range(len(saved_data[0]['model_outputs'])):
        
    model_match_answer = match_answer(saved_data[0]['model_outputs'][response_id])
    
    if model_match_answer[0]:
        extracted_model_output = model_match_answer[1]
    else:
        extracted_model_output = ""
    
    is_correct = False
    # grade simple algebra questions. if succeeded, return; otherwise, proceed to more complex grading
    if grade_answer(extracted_model_output, ground_truth):
        is_correct = True
    
    else:
        try:
            if "\\pi" in extracted_model_output or "\\pi" in ground_truth:
                equivs = []
                for pi in [math.pi, 3.14]:
                    equivs.append(math_equal(extracted_model_output, ground_truth, timeout=True, pi=pi))
                is_correct = any(equivs)
            else:
                is_correct = math_equal(extracted_model_output, ground_truth, timeout=True)
        except:
            is_correct = False
    
    
    if not is_correct:
        # use LLM to grade the answer
        inputs = input_template.replace("[QUESTION]", saved_data[0]['question']).replace("[STUDENT'S ANSWER]", extracted_model_output).replace("[REFERENCE ANSWER]", ground_truth)
        data = {
            "model": "Qwen/Qwen2.5-7B-Instruct",
            "messages": [{"role": "user", "content": inputs}],
        }
        response = requests.post(url, json=data)
        
        llm_is_correct = not "INCORRECT" in response.json()['choices'][0]['message']['content']
        if llm_is_correct:
            print("!")
    print(is_correct, extracted_model_output, ground_truth)

False 0.5 [0, \\frac{1}{2}]
True \left[0, \frac{1}{2}\right] [0, \\frac{1}{2}]
True \left[0, \frac{1}{2}\right] [0, \\frac{1}{2}]
False (-\infty, \infty) [0, \\frac{1}{2}]
False \emptyset [0, \\frac{1}{2}]
False \left( \frac{1}{2}, 1 \right) [0, \\frac{1}{2}]
!
False \left(0, \frac{1}{2}\right) [0, \\frac{1}{2}]
False [0.5, 1] [0, \\frac{1}{2}]
True \left[0, \dfrac{1}{2}\right] [0, \\frac{1}{2}]
False [0, 1] [0, \\frac{1}{2}]
False \left[ \frac{1}{2}, 1 \right] [0, \\frac{1}{2}]
False \emptyset [0, \\frac{1}{2}]
False  [0, \\frac{1}{2}]
True [0, 0.5] [0, \\frac{1}{2}]
False [0.5, \infty) [0, \\frac{1}{2}]
True \left[0, \dfrac{1}{2}\right] [0, \\frac{1}{2}]
False \emptyset [0, \\frac{1}{2}]
False \varnothing [0, \\frac{1}{2}]
False \left[ \frac{1}{2}, 1 \right] [0, \\frac{1}{2}]
False \emptyset [0, \\frac{1}{2}]
False \varnothing [0, \\frac{1}{2}]
False \emptyset [0, \\frac{1}{2}]
False \varnothing [0, \\frac{1}{2}]
True [0, \frac{1}{2}] [0, \\frac{1}{2}]
False \left(0, \dfrac{1}{2}\rig

In [36]:
print(saved_data[0]['question'])

Given $p$: $|4x-3|\leqslant 1$ and $q$: $x^{2}-(2a+1)x+a^{2}+a\leqslant 0$, find the range of values for $a$ if $p$ is a necessary but not sufficient condition for $q$.


In [33]:
print(saved_data[0]['model_outputs'][0])

Alright, so I've got this problem here where I need to find the range of values for 'a' such that the condition p is a necessary but not sufficient condition for q. Let me try to unpack what this means step by step.

First, let's understand what p and q are.

Condition p is given by |4x - 3| ≤ 1. That looks like an absolute value inequality, which I think means that 4x - 3 is between -1 and 1. So, I can rewrite this as:

-1 ≤ 4x - 3 ≤ 1

To solve for x, I can add 3 to all parts of the inequality:

-1 + 3 ≤ 4x ≤ 1 + 3

Which simplifies to:

2 ≤ 4x ≤ 4

Then, divide each part by 4:

0.5 ≤ x ≤ 1

So, condition p is true when x is between 0.5 and 1, inclusive.

Now, condition q is given by the quadratic inequality:

x² - (2a + 1)x + a² + a ≤ 0

Hmm, okay. So, this is a quadratic in terms of x. Let me think about how to solve this inequality. First, I should probably factor it or find its roots.

Let me write it down again:

x² - (2a + 1)x + a² + a ≤ 0

I can try factoring this quadratic. L

In [32]:
model_match_answer = match_answer(saved_data[0]['model_outputs'][0])
print(model_match_answer)

(True, '0.5')
